# Import libraries

In [2]:
import numpy as np

# Binomial trees generic function

In [3]:
def binomialTree(
    So: float = 100, 
    K: float = 100, 
    t: int = 4, 
    option_type: str = "C", 
    E: int = 1, 
    interest_rate: float = 1, 
    vol: float = 0.2, 
    exercise_type: str = "EUR"
    ) -> dict:

    '''
    Evaluating binomial trees for options. The function returns a dictionary with indexes:
    'S': Stock prices path;
    'P': Option payoffs;
    'O': Option prices;
    'D': Delta hedge values.

    So: float.
    Stock price today.

    K: float.
    Strike of the option.

    t: integer.
    Time steps for expiration.

    option_type: string.
    Option type. ("C" for Call and "P" for Put)

    E: integer.
    Expiration days.

    interest_rate: float.
    Interest rate.

    vol: float.
    Volatility.

    exercise_type: string.
    Type of exercise. ("US" for American and "EUR" for European)
    '''

    Su, Sv = So, So
    
    prices_matrix = np.zeros((t + 1, t + 1))
    option_price_matrix = np.zeros((t + 1, t + 1))
    payoff_matrix = np.zeros((t + 1, t + 1))
    delta_hedge_matrix = np.zeros((t + 1, t + 1))

    dt = E / t
    discount_factor = 1 / (1 + interest_rate * dt)
    prob = 0.5 + interest_rate * np.sqrt(dt) / (2 * vol)
    payoff_type = 1 if option_type == "C" else -1
    early_exercise = 1 if exercise_type == "US" else 0
    v = 1 + vol * np.sqrt(dt)
    u = 1 - vol * np.sqrt(dt)

    for col in range(t + 1):
        for idx in range(col + 1):
            prices_matrix[0][col] = Sv
            prices_matrix[col][col] = Su
            payoff_matrix[0][col] = max(payoff_type * (prices_matrix[0][col] - K), 0)
            payoff_matrix[col][col] = max(payoff_type * (prices_matrix[col][col] - K), 0)
            
            if idx < len(prices_matrix[0]) - 1 and col > 1 and idx < col:
                prices_matrix[idx][col] = prices_matrix[idx][col - 1] * v
                prices_matrix[idx + 1][col] = prices_matrix[idx - 1][col - 1] * u
                payoff_matrix[idx][col] = max(payoff_type * (prices_matrix[idx][col] - K), 0)
        Sv = Sv * v
        Su = Su * u
            
    for col in range(t, -1, -1):
        for idx in range(col, -1, -1):
            if col == t:
                option_price_matrix[idx][col] = max(payoff_type * (prices_matrix[idx][col] - K), early_exercise * max(payoff_type * (prices_matrix[idx][col] - K), 0))
            else:
                if idx < t and col < t:
                    option_price_matrix[idx][col] = max(discount_factor * (prob * option_price_matrix[idx][col + 1] + (1 - prob) * option_price_matrix[idx + 1][col + 1]), early_exercise * max(payoff_type * (prices_matrix[idx][col] - K), 0))
                    delta_hedge_matrix[idx][col] = (option_price_matrix[idx][col + 1] - option_price_matrix[idx + 1][col + 1])/(prices_matrix[idx][col + 1] - prices_matrix[idx + 1][col + 1])

    delta_hedge_matrix[:, -1] = 0

    data_dict = {
        "S": np.around(prices_matrix, 2),
        "P": np.around(payoff_matrix, 2),
        "O": np.around(option_price_matrix, 2),
        "D": np.around(delta_hedge_matrix, 2)
    }

    return data_dict

## Binomial trees function description

In [4]:
binomialTree?

Signature:
binomialTree(
    So: float = 100,
    K: float = 100,
    t: int = 4,
    option_type: str = 'C',
    E: int = 1,
    interest_rate: float = 1,
    vol: float = 0.2,
    exercise_type: str = 'EUR',
) -> dict
Docstring:
Evaluating binomial trees for options. The function returns a dictionary with indexes:
'S': Stock prices path;
'P': Option payoffs;
'O': Option prices;
'D': Delta hedge values.

So: float.
Stock price today.

K: float.
Strike of the option.

t: integer.
Time steps for expiration.

option_type: string.
Option type. ("C" for Call and "P" for Put)

E: integer.
Expiration days.

interest_rate: float.
Interest rate.

vol: float.
Volatility.

exercise_type: string.
Type of exercise. ("US" for American and "EUR" for European)
File:      c:\users\angelo\appdata\local\temp\ipykernel_6116\2061377885.py
Type:      function


## Binomial trees example

In [5]:
binomialResults = binomialTree(
    So = 100,
    K = 100,
    t = 4,
    option_type = 'C',
    E = 1,
    interest_rate = 0.05,
    vol = 0.2,
    exercise_type = 'EUR'
)

In [6]:
print(f"European Call Option Price using Binomial Tree Method is {binomialResults['O'][0][0]:.2f}")

European Call Option Price using Binomial Tree Method is 10.29
